In [129]:
import math
from numpy import pi, deg2rad, rad2deg
from modsim import *
import pandas as pd
# install Pint if necessary

try:
    from pint import UnitRegistry
except ImportError:
    !pip install pint
    
# import units
from pint import UnitRegistry
units = UnitRegistry()

Constants

In [135]:
# baseball mass (kg)
BASEBALL_MASS = 0.145

# baseballRadius (cm)
BASEBALL_RADIUS = 3.683

# air density (kg/m^3)
P = 1.225

# g
G = 9.80665

# drag coefficient
Cd = 0.33

CL = 0.22


Algos

In [126]:
# baseball cross-sectional area
def CSA(r):
    CROSS_SECTIONAL_AREA = 0.25 * math.pi * (2 * radius) ** 2
    CROSS_SECTIONAL_AREA_M2 = CROSS_SECTIONAL_AREA / 100 ** 2
    return CROSS_SECTIONAL_AREA_M2


# Converting miles per hour to meters per second.
def msec(mph):
    msec = mph * (1609/3600)
    return msec

# Converting meters per second to miles per hour.
def mphsec(msec):
    mphsec = msec * 2.237
    return mphsec

# Defining a function that takes a mass and returns the gravitational force.
def GRAV_FORCE(m):
    GRAV_FORCE = m * G
    GRAV_FORCE

# Defining a function that takes a value in RPM and returns the value in rad/s.
def RAD_SEC(rpm):
    RAD_SEC = rpm * 2 * math.pi / 60
    return RAD_SEC

# This function is taking the rotational speed of the baseball and the velocity of the baseball and returning the spin factor.
def S(spin):
    spin = radsec / v0
    return spin

# This function is taking the rotational speed of the baseball and the velocity of the baseball and returning the spin factor in meters/sec.
def vr_msec(rpm):
    radsec = RAD_SEC(rpm)
    v = BASEBALL_RADIUS * radsec / 100
    return v

# Taking the radius of the baseball and the rotational speed of the baseball and returning the tangential speed of the baseball.
def tangential_speed(radius, rpm):
    v = radius * rotational_speed(rpm)
    return v

# Converting the tangential velocity from meters per second to miles per hour.
def vr_mph(rpm):
    radsec = RAD_SEC(rpm)
    v = BASEBALL_RADIUS * radsec / 100
    v = mphsec(v)
    return v

# This function is taking the rotational speed of the baseball and the velocity of the baseball and returning the spin factor.
def spin_factor(rpm, velocity):
    spin_factor = rotational_speed(rpm) / msec(velocity)
    return f'The spin factor of {rpm}rpm and {velocity}mph is {round(spin_factor, 4)}'

In [141]:
# Throw parameters
# x0 = feet; y0 = release point of hand in meters; v0 = release speed in meters/sec
mph = 90
x0 = 0
y0 = 1.75
v0 = msec(mph)
rpm = 2200
radsec = RAD_SEC(rpm)
angle = 45
time = 1
top = 0

In [142]:
# drag_force = - 0.5 * Cd * density * area * velocity ** 2  
# (in Newtons in direction opposite to current velocity)
drag_force = -0.5 * Cd * P * CSA(BASEBALL_RADIUS) * v0 ** 2
drag_force

-1.393687641180017

In [131]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

In [132]:
# drag coefficient
download('https://github.com/AllenDowney/ModSim/raw/main/data/' +
         'baseball_drag.csv')

baseball_drag = pd.read_csv('baseball_drag.csv')

mph_to_mps = (1 * units.mph).to(units.m/units.s).magnitude

speed = baseball_drag['Velocity in mph'] * mph_to_mps

C_d_series = make_series(speed, baseball_drag['Drag coefficient'])

drag_interp = interpolate(C_d_series)

array(0.31504741)

In [133]:
# Defining a function that takes a value of msec and returns the value of Cd at that point.
def Cd(msec):
    Cd = drag_interp(msec)
    return Cd

In [160]:
def throw(mph, rpm, angle, top):
    if top == 1:
        top = top * -1
    # Interpolating the drag coefficient for a baseball at v0 in m/s.
    Cd = drag_interp(msec(mph))
    # Calculating drag force
    drag_force = -0.5 * Cd * P * CSA(BASEBALL_RADIUS) * msec(mph) ** 2
    # Calculating the magnus force
    magnus = 0.5 * CL * P * CSA(BASEBALL_RADIUS) * msec(mph) ** 2
    # Calculating the ratio of the rotational speed of the ball to the translational speed of the ball.
    spin_factor = rotational_speed(rpm) / msec(mph)
    # Calculating the x component of the acceleration of the ball.
    ax = -1 * drag_force * math.cos(angle) - magnus * math.sin(angle)
    # Calculating the y component of the acceleration of the ball.
    ay = magnus * math.cos(angle) - drag_force * math.sin(angle) - G
    print(f'magnus: {magnus}')
    print(f'drag force: {drag_force}')
    print(f'Cd: {Cd}')
    print(f'ax: {ax}')
    print(f'ay: {ay}')
    print(f'spin factor: {spin_factor}')
    print(f'gravity: {G}')
    
    

In [161]:
throw(90, 2200, 45, 0)

magnus: 0.9291250941200111
drag force: -1.402946317385928
Cd: 0.33219228689246594
ax: -0.05359726766612638
ay: -8.124788191500567
spin factor: 0.21093903985893228
gravity: 9.80665


In [ ]:
# http://www.physics.usyd.edu.au/~cross/TRAJECTORIES/42.%20Ball%20Trajectories.pdf

In [ ]:
vx = -CSA(BASEBALL_RADIUS)*msec(mph)*((Cd * msec(mph)) + (CL * ))